In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = "your role"

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

# hyperparameters, which are passed into the training job
hyperparameters={
    'teacher_id':'mariagrandury/roberta-base-finetuned-sms-spam-detection',           
    'student_id':'nreimers/MiniLMv2-L12-H384-distilled-from-RoBERTa-Large',           
    'dataset_id':'sms_spam',           
    'dataset_config':'plain_text',             
    'epochs': 6,             
    # distillation parameter
    'alpha': 0.055199695773231194, # 0.5,
    'temparature': 19, # 4 
    'learning_rate': 1e-4, # 3e-5
    # hpo parameter
    "run_hpo": False,
    "n_trials": 100,   
    # push to hub config
    'push_to_hub': True,                            
    'hub_model_id': 'MiniLMv2-L12-H384-distilled-finetuned-spam-detection', 
    'hub_token': ""    
}

# define Training Job Name 
job_name = f'knowledge-distillation'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        
    source_dir           = './scripts',       
    instance_type        = 'ml.p2.xlarge',   
    instance_count       = 1,                 
    role                 = role,    
    base_job_name        = job_name, 
    transformers_version = '4.17',            
    pytorch_version      = '1.10',             
    py_version           = 'py38',            
    hyperparameters      = hyperparameters,   
)

In [ ]:
huggingface_estimator.fit(wait=False)

In [ ]:
predictor =huggingface_estimator.deploy(1,"ml.c5.xlarge")

In [ ]:
sentiment_input= {"inputs": "your input"}

for i in range(1000):
    predictor.predict(sentiment_input)

In [ ]:
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#metricsV2:graph=~(metrics~(~(~'AWS*2fSageMaker~'ModelLatency~'EndpointName~'knowledge-distillation-2022-06-02-13-25-41-341~'VariantName~'AllTraffic))~view~'timeSeries~stacked~false~region~'us-east-1~start~'-PT1H~end~'P0D~stat~'p99~period~30);query=~'*7bAWS*2fSageMaker*2cEndpointName*2cVariantName*7d*20

In [ ]:
predictor =huggingface_estimator.deploy(1,"ml.p2.xlarge")